# A notebook for creating sc graph animations with glimpses

In [9]:
import napari
from cellpose import models
from octopuslite import utils, tile
import numpy as np
import os

def view(img):
    return napari.Viewer().add_image(img)
from napari_animation import Animation
from tqdm.auto import tqdm

import btrack
import dask.array as da

from pathlib import Path
import cv2

from skimage.transform import rescale, resize, downscale_local_mean
from skimage.io import imsave, imread
from scipy.ndimage.filters import gaussian_filter1d

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style="ticks")
sns.set_palette("rocket_r")

def msd_calc(x1, y1, x2, y2):
    """
    Displacement calculation for cell movement between frames
    """
    return np.sqrt((x1-x2)**2+(y1-y2)**2)

def update_slider(event):
    # only trigger if update comes from first axis (optional)
        #ind_lambda = viewer.dims.indices[0]
    time = viewer.dims.current_step[0]
    viewer.text_overlay.text = f"{time:1.1f} hours"
text_size = 24
napari_scale = [1.4949402023919043e-07, 1.4949402023919043e-07]

import glob
from natsort import natsorted

label_text_size = 15
### glimpse size
size = 500
### resized images are to this scale
scale = 6048/1200

import os
import cv2
from natsort import natsorted

import sys 
sys.path.append('../')

from macrohet import tools# import compile_mp4, compile_multi_track_df



/tmp/ipykernel_330135/948269833.py:20: DeprecationWarning: Please use `gaussian_filter1d` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  from scipy.ndimage.filters import gaussian_filter1d


In [31]:

import importlib
importlib.reload(tools)

<module 'macrohet.tools' from '/home/dayn/analysis/macrohet/track_analysis/../macrohet/tools.py'>

### Load all metadata

Both the image metadata and the assay layout metadata

In [4]:
image_dir = '/mnt/DATA/sandbox/pierre_live_cell_data/outputs/Replication_IPSDM_GFP/Images/'
image_metadata_fn = '/mnt/DATA/sandbox/pierre_live_cell_data/outputs/Replication_IPSDM_GFP/Index.idx.xml'
metadata = utils.read_harmony_metadata(image_metadata_fn)
assay_layout_metadata = '/mnt/DATA/sandbox/pierre_live_cell_data/outputs/Replication_IPSDM_GFP/Assaylayout/20210602_Live_cell_IPSDMGFP_ATB.xml'
assay_layout = utils.read_harmony_metadata(assay_layout_metadata, assay_layout=True)
assay_layout

Reading metadata XML file...


Extracting HarmonyV5 metadata:   0%|          | 0/113400 [00:00<?, ?it/s]

Extracting metadata complete!
Reading metadata XML file...
Extracting metadata complete!


Strain Compound Concentration ConcentrationEC
3 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA            60            EC50
  7     WT      RIF           0.1            EC50
  8     WT      INH          0.04            EC50
  9     WT      BDQ          0.02            EC50
4 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA            60            EC50
  7     WT      RIF           0.1            EC50
  8     WT      INH          0.04            EC50
  9     WT      BDQ          0.02            EC50
5 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA           400            EC99
  7     WT      RIF             2            EC99
  8     WT      INH             2            EC99
  9     WT      BDQ           2.5            EC99
6 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA           400            EC99
  7     WT      RIF             2            EC99
  8     WT      INH             2            EC99
  9     WT      BDQ           2.5            EC99

# Iteratively load all tracks

and append to a track_dict dictionary

In [6]:
tracks_dict = dict()
segmentation_dict = dict()
### iterate over all experimental conditions
for (row, column), info in tqdm(assay_layout.iterrows(), 
                                desc = 'Progress through positions',
                                total = len(assay_layout)):
    ### load tracks
    with btrack.dataio.HDF5FileHandler(
            f'/mnt/DATA/macrohet/segmentation/full_localisation/({row},{column}).h5',#
            'r', 
            obj_type = 'obj_type_1', 
            ) as hdf: 
            tracks = hdf.tracks
            seg = hdf.segmentation
            
    tracks_dict[(row, column)] = tracks
    segmentation_dict[(row, column)] = seg


Progress through positions:   0%|          | 0/24 [00:00<?, ?it/s]

[INFO][2023/03/10 04:16:35 PM] Opening HDF file: /mnt/DATA/macrohet/segmentation/full_localisation/(3,4).h5...
[INFO][2023/03/10 04:16:35 PM] Loading tracks/obj_type_1
[INFO][2023/03/10 04:16:35 PM] Loading LBEP/obj_type_1
[INFO][2023/03/10 04:16:35 PM] Loading objects/obj_type_1 (25554, 5) (25554 filtered: None)
[INFO][2023/03/10 04:16:36 PM] Loading segmentation (75, 1200, 1200)
[INFO][2023/03/10 04:16:36 PM] Closing HDF file: /mnt/DATA/macrohet/segmentation/full_localisation/(3,4).h5
[INFO][2023/03/10 04:16:36 PM] Opening HDF file: /mnt/DATA/macrohet/segmentation/full_localisation/(3,5).h5...
[INFO][2023/03/10 04:16:36 PM] Loading tracks/obj_type_1
[INFO][2023/03/10 04:16:36 PM] Loading LBEP/obj_type_1
[INFO][2023/03/10 04:16:36 PM] Loading objects/obj_type_1 (26917, 5) (26917 filtered: None)
[INFO][2023/03/10 04:16:37 PM] Loading segmentation (75, 1200, 1200)
[INFO][2023/03/10 04:16:37 PM] Closing HDF file: /mnt/DATA/macrohet/segmentation/full_localisation/(3,5).h5
[INFO][2023/03/1

[INFO][2023/03/10 04:16:57 PM] Closing HDF file: /mnt/DATA/macrohet/segmentation/full_localisation/(5,8).h5
[INFO][2023/03/10 04:16:57 PM] Opening HDF file: /mnt/DATA/macrohet/segmentation/full_localisation/(5,9).h5...
[INFO][2023/03/10 04:16:57 PM] Loading tracks/obj_type_1
[INFO][2023/03/10 04:16:57 PM] Loading LBEP/obj_type_1
[INFO][2023/03/10 04:16:57 PM] Loading objects/obj_type_1 (22122, 5) (22122 filtered: None)
[INFO][2023/03/10 04:16:57 PM] Loading segmentation (75, 1200, 1200)
[INFO][2023/03/10 04:16:57 PM] Closing HDF file: /mnt/DATA/macrohet/segmentation/full_localisation/(5,9).h5
[INFO][2023/03/10 04:16:57 PM] Opening HDF file: /mnt/DATA/macrohet/segmentation/full_localisation/(6,4).h5...
[INFO][2023/03/10 04:16:57 PM] Loading tracks/obj_type_1
[INFO][2023/03/10 04:16:57 PM] Loading LBEP/obj_type_1
[INFO][2023/03/10 04:16:57 PM] Loading objects/obj_type_1 (24744, 5) (24744 filtered: None)
[INFO][2023/03/10 04:16:58 PM] Loading segmentation (75, 1200, 1200)
[INFO][2023/03/1

# Compile all full length tracks into dataframe

Add extra information such as the MSD of cells between frames

In [20]:
df = tools.compile_multi_track_df(tracks_dict, assay_layout, track_len = 75)
df

,Time (hours),x,y,Area,Intracellular Mtb content,Mean Mtb content,Macroph. GFP expression,Eccentricity,MSD,Strain,Compound,Concentration,Cell ID,Acquisition ID
0,0,1169.847412,13.893421,760.0,0.005136,0.005272,0.014705,0.809516,0.000000,RD1,CTRL,EC0,343,"(3, 4)"
1,1,1172.826538,14.674929,1049.0,0.005121,0.005272,0.013235,0.843927,3.079926,RD1,CTRL,EC0,343,"(3, 4)"
2,2,1174.588867,14.375803,934.0,0.005116,0.005272,0.013899,0.719524,1.787535,RD1,CTRL,EC0,343,"(3, 4)"
3,3,1175.250610,12.160207,774.0,0.005094,0.005272,0.013428,0.739700,2.312308,RD1,CTRL,EC0,343,"(3, 4)"
4,4,1180.553955,9.897770,538.0,0.005071,0.005272,0.012774,0.761300,5.765768,RD1,CTRL,EC0,343,"(3, 4)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81520,70,559.663025,1177.240234,3018.0,0.005635,0.005657,0.024949,0.698306,3.245095,WT,BDQ,EC99,155,"(6, 9)"
81521,71,556.457581,1178.115601,2863.0,0.005609,0.005657,0.024090,0.716728,3.322821,WT,BDQ,EC99,155,"(6, 9)"
81522,72,559.905823,1178.514648,3047.0,0.005600,0.005657,0.022092,0.789115,3.471255,WT,BDQ,EC99,155,"(6, 9)"
81523,73,565.014221,1180.512939,2737.0,0.005606,0.005657,0.021206,0.822009,5.485335,WT,BDQ,EC99,155,"(6, 9)"


### Add category to discern initial amount of Mtb growth

Categorical

In [21]:
initial_mtb = df.loc[df['Time (hours)'] == 0, 'Intracellular Mtb content']
initial_mtb_quartiles = pd.cut(initial_mtb, bins = [initial_mtb.quantile(.0), 
                                                    initial_mtb.quantile(.25), 
                                                    initial_mtb.quantile(0.5), 
                                                    initial_mtb.quantile(.75),
                                                    initial_mtb.quantile(1)], 
                                                    labels = ['Lower', 'Lower-mid', 'Upper-mid', 'Upper'])
df['Initial Mtb load (quartile)'] = initial_mtb_quartiles
df.fillna(method='ffill', inplace=True)
final_mtb = df.loc[df['Time (hours)'] == 74, 'Intracellular Mtb content']
final_mtb_quartiles = pd.cut(final_mtb, bins = [final_mtb.quantile(.0), 
                                                    final_mtb.quantile(.25), 
                                                    final_mtb.quantile(0.5), 
                                                    final_mtb.quantile(.75),
                                                    final_mtb.quantile(1)], 
                                                    labels = ['Lower', 'Lower-mid', 'Upper-mid', 'Upper'])
df['Final Mtb load (quartile)'] = final_mtb_quartiles
df.fillna(method='bfill', inplace=True)

Now continuous assessment: ie. the actual value of the Mtb content in the first frame

In [22]:
initial_mtb = df.loc[df['Time (hours)'] == 0, 'Intracellular Mtb content']
df['Initial Mtb load'] = initial_mtb
df.fillna(method='ffill', inplace=True)

In [23]:
final_mtb = df.loc[df['Time (hours)'] == 74, 'Intracellular Mtb content']
df['Final Mtb load'] = final_mtb
df.fillna(method='bfill', inplace=True)

## Pick a single experiment to focus on?

In [24]:
### select from conditions
selected_expt_df = df[(df['Compound'] == 'PZA') & (df['Concentration'] == 'EC50')]
acq_ID = selected_expt_df['Acquisition ID'].iloc[0]

In [41]:
### get extreme cells
extreme_cases = pd.concat([selected_expt_df[(selected_expt_df['Mean Mtb content'] == np.max(selected_expt_df['Mean Mtb content']))],
                           selected_expt_df[(selected_expt_df['Mean Mtb content'] == np.min(selected_expt_df['Mean Mtb content']))]])
extreme_cases

,Time (hours),x,y,Area,Intracellular Mtb content,Mean Mtb content,Macroph. GFP expression,Eccentricity,MSD,Strain,Compound,Concentration,Cell ID,Acquisition ID,Initial Mtb load (quartile),Final Mtb load (quartile),Initial Mtb load,Final Mtb load
15300,0,268.956024,65.425865,2637.0,0.006256,0.007892,0.016216,0.829670,0.000000,WT,INH,EC50,300,"(3, 8)",Upper,Upper,0.006256,0.010684
15301,1,272.280609,60.727016,3535.0,0.005994,0.007892,0.014438,0.729552,5.756044,WT,INH,EC50,300,"(3, 8)",Upper,Upper,0.006256,0.010684
15302,2,279.796692,46.732731,3330.0,0.005973,0.007892,0.013204,0.683020,15.884947,WT,INH,EC50,300,"(3, 8)",Upper,Upper,0.006256,0.010684
15303,3,279.816193,53.272476,3237.0,0.005982,0.007892,0.013640,0.807284,6.539774,WT,INH,EC50,300,"(3, 8)",Upper,Upper,0.006256,0.010684
15304,4,279.036102,52.219032,3657.0,0.005994,0.007892,0.013985,0.625249,1.310834,WT,INH,EC50,300,"(3, 8)",Upper,Upper,0.006256,0.010684
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17395,70,437.507751,677.482788,1160.0,0.005289,0.005293,0.019831,0.576796,5.990601,WT,INH,EC50,81,"(3, 8)",Lower,Lower,0.005292,0.005288
17396,71,435.135040,678.725037,1022.0,0.005291,0.005293,0.020450,0.603804,2.678234,WT,INH,EC50,81,"(3, 8)",Lower,Lower,0.005292,0.005288
17397,72,439.116241,679.119019,1084.0,0.005295,0.005293,0.021352,0.524047,4.000648,WT,INH,EC50,81,"(3, 8)",Lower,Lower,0.005292,0.005288
17398,73,440.520416,680.639954,1159.5,0.005292,0.005293,0.019682,0.627327,2.070012,WT,INH,EC50,81,"(3, 8)",Lower,Lower,0.005292,0.005288


In [26]:
### isolate a single cell ID from df of interesting cells
ID = 300
### isolate sc df
sc_df = extreme_cases[extreme_cases['Cell ID']==ID]

In [42]:
### or manually select from acquisition ID and Cell ID 
acq_ID = (3, 8)
selected_expt_df = df[(df['Acquisition ID'] == acq_ID)]
### get cell ID
ID = 81
### isolate sc df
sc_df = selected_expt_df[selected_expt_df['Cell ID']==ID]

In [43]:
sc_df

,Time (hours),x,y,Area,Intracellular Mtb content,Mean Mtb content,Macroph. GFP expression,Eccentricity,MSD,Strain,Compound,Concentration,Cell ID,Acquisition ID,Initial Mtb load (quartile),Final Mtb load (quartile),Initial Mtb load,Final Mtb load
17325,0,443.839935,660.719543,1462.0,0.005292,0.005293,0.017062,0.931162,0.000000,WT,INH,EC50,81,"(3, 8)",Lower,Lower,0.005292,0.005288
17326,1,442.930603,667.374207,1729.0,0.005300,0.005293,0.016790,0.927155,6.716504,WT,INH,EC50,81,"(3, 8)",Lower,Lower,0.005292,0.005288
17327,2,444.404968,664.894165,1531.0,0.005269,0.005293,0.014994,0.917178,2.885196,WT,INH,EC50,81,"(3, 8)",Lower,Lower,0.005292,0.005288
17328,3,446.667236,652.623169,1794.0,0.005303,0.005293,0.017638,0.933294,12.477788,WT,INH,EC50,81,"(3, 8)",Lower,Lower,0.005292,0.005288
17329,4,445.565918,651.952576,1350.0,0.005273,0.005293,0.015478,0.932637,1.289417,WT,INH,EC50,81,"(3, 8)",Lower,Lower,0.005292,0.005288
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17395,70,437.507751,677.482788,1160.0,0.005289,0.005293,0.019831,0.576796,5.990601,WT,INH,EC50,81,"(3, 8)",Lower,Lower,0.005292,0.005288
17396,71,435.135040,678.725037,1022.0,0.005291,0.005293,0.020450,0.603804,2.678234,WT,INH,EC50,81,"(3, 8)",Lower,Lower,0.005292,0.005288
17397,72,439.116241,679.119019,1084.0,0.005295,0.005293,0.021352,0.524047,4.000648,WT,INH,EC50,81,"(3, 8)",Lower,Lower,0.005292,0.005288
17398,73,440.520416,680.639954,1159.5,0.005292,0.005293,0.019682,0.627327,2.070012,WT,INH,EC50,81,"(3, 8)",Lower,Lower,0.005292,0.005288


# First assess glimpse with scale bars etc

In [65]:
importlib.reload(tools)

<module 'macrohet.tools' from '/home/dayn/analysis/macrohet/track_analysis/../macrohet/tools.py'>

In [67]:
### use acq ID to pre load correct images
row, column = acq_ID
images = tile.compile_mosaic(
                             image_dir, 
                             metadata, 
                             row, 
                             column, 
                             set_plane = 'sum_proj',
                             ).astype(np.uint16)
### create base dirname
basedir = f'/mnt/DATA/macrohet/results/glimpses/labelled/{acq_ID}/{ID}/'
Path(os.path.dirname(basedir)).mkdir(parents=True, exist_ok=True)
# ### make glimpse stack of images
glimpse_stack = tools.create_glimpse_from_sc_df(sc_df, acq_ID, ID, images)#np.stack(glimpse_stack, axis = 1)
### load glimpse into memory
print(f'Loading glimpse stack {acq_ID, ID} into memory (can take several minutes)')
glimpse_stack = glimpse_stack.compute().compute()

### make glimpse stack of masks
print(f'Creating mask glimpse ID: {acq_ID, ID}')
mask_glimpse_stack, mask_shapes = create_mask_glimpse_from_sc_df(sc_df, acq_ID, ID, masks,)
### get expt info
expt_info = sc_df['Strain'].iloc[0],sc_df['Compound'].iloc[0],sc_df['Concentration'].iloc[0], sc_df['Cell ID'].iloc[0]
expt_info = ','.join(map(str, expt_info))
### create a set of points to anchor labels at
label_points = [[t, 0, 500] for t in sc_df['Time (hours)']]
### create labels
text_overlay = f'cell ID:{acq_ID[0], acq_ID[1], ID}'
fixed_labels = {'macrophage':['Macrophage:green' for i in range(len(mask_shapes))],
                'mtb':['Mtb:magenta' for i in range(len(mask_shapes))],
                'mask':[f'Mask:cyan' for i in range(len(mask_shapes))],
                'info':[f'ID:{expt_info}' for i in range(len(mask_shapes))]}
### create lists of labels for tidier iteration of adding labels 
labels_list = list([fixed_labels['macrophage'], fixed_labels['mtb'], fixed_labels['mask'], fixed_labels['info']])
offset_list = list([[20,-5], [40,-5], [60,-5], [80,-5]])
colour_list = list(['#39FF14', 'magenta', 'cyan', 'white'])

### new way of checking with overlaid labels etc
viewer = napari.Viewer()
viewer.add_image(mask_glimpse_stack, scale = napari_scale, colormap='cyan')
viewer.add_image(glimpse_stack, channel_axis = 0, colormap= ['green', 'magenta'], blending = ['additive', 'additive'],
                 scale = napari_scale)
viewer.theme = 'light'
viewer.scale_bar.visible = True
viewer.scale_bar.unit = 'm'
viewer.scale_bar.font_size = text_size
viewer.text_overlay.visible = True
viewer.text_overlay.color = 'black'
viewer.text_overlay.position = 'bottom_left'
viewer.text_overlay.font_size = text_size
viewer.dims.events.current_step.connect(update_slider)

for config in zip(labels_list, offset_list, colour_list):
    string, offset, colour = config
    viewer.add_points(
    label_points, 
    scale = [1, napari_scale[0], napari_scale[1]],
    face_color = 'transparent', 
    edge_color = 'transparent', 
    text = {'string':string,
            'anchor': 'upper_right',
            'translation': offset,
            'size': text_size,
            'color': colour}, )

Creating glimpse ID: ((3, 8), 81):   0%|          | 0/75 [00:00<?, ?it/s]

/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/dask/array/core.py:1744: PerformanceWarning: Increasing number of chunks by factor of 23
  return da_func(*args, **kwargs)


Loading glimpse stack ((3, 8), 81) into memory (can take several minutes)
Creating mask glimpse ID: ((3, 8), 81)


Creating mask glimpse ID: ((3, 8), 81):   0%|          | 0/75 [00:00<?, ?it/s]

/tmp/ipykernel_330135/3847468708.py:45: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  mask_shapes = np.asarray(coords_stack, )#axis = 0)
/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari_tools_menu/__init__.py:194: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


# Check view 

In [70]:
time_overlay_font_size = 26
scale_bar_font_size = 26
label_font_size = 30
text_colour = 'white'

In [75]:
palette = sns.color_palette('PiYG')

In [73]:
zoom = viewer.camera.zoom
cam_coords = viewer.camera.center
print(zoom, cam_coords)
### use this to assess camera angles before proceeding

13243339.076923076 (0.0, 3.7298758059023385e-05, 3.7298758059023385e-05)


In [253]:
viewer.theme = 'dark'
viewer.scale_bar.colored = True
viewer.scale_bar.color = text_colour

In [76]:
### create base dirname
basedir = f'/mnt/DATA/macrohet/results/glimpses/labelled/multi_plot/{acq_ID}/{ID}/'
Path(os.path.dirname(basedir)).mkdir(parents=True, exist_ok=True)

# Then create .mp4 of glimpse and animated graphs

In [77]:
### save glimpse as a series of images
print(f'Creating glimpse image sequence ID: {acq_ID, ID}')
### create output directory
glimpse_seq_basedir = os.path.join(basedir, f'{ID}_glimpse_seq')
Path(glimpse_seq_basedir).mkdir(parents=True, exist_ok=True)
### create napari instances and use to save glimpse frames with scale bar and time
viewer = napari.Viewer()
viewer.add_image(mask_glimpse_stack, scale = napari_scale, colormap='cyan')
viewer.add_image(glimpse_stack, channel_axis = 0, colormap= ['green', 'magenta'], blending = 'additive', scale = napari_scale)
viewer.theme = 'dark'
viewer.scale_bar.visible = True
viewer.scale_bar.unit = 'm'
viewer.scale_bar.font_size = scale_bar_font_size
viewer.scale_bar.colored = True
viewer.scale_bar.color = text_colour
viewer.scale_bar.ticks = False
viewer.text_overlay.visible = True
viewer.text_overlay.color = text_colour
viewer.text_overlay.position = 'bottom_left'
viewer.text_overlay.font_size = time_overlay_font_size
viewer.dims.events.current_step.connect(update_slider)
for config in zip(labels_list, offset_list, colour_list):
    string, offset, colour = config
    viewer.add_points(
    label_points, 
    scale = [1, napari_scale[0], napari_scale[1]],
    face_color = 'transparent', 
    edge_color = 'transparent', 
    text = {'string':string,
            'anchor': 'upper_right',
            'translation': offset,
            'size': label_font_size,
            'color': colour}, )
### save image sequence
for t in tqdm(list(sc_df['Time (hours)']), desc = f'Saving glimpse image sequence ID: {acq_ID, ID}'):
    viewer.dims.current_step = (t, cam_coords[-2], cam_coords[-1])
    viewer.camera.zoom = zoom*0.95
    glimpse_seq_fn = os.path.join(glimpse_seq_basedir, f'glimpse_{ID}_t_{t}.tiff')
    imsave(glimpse_seq_fn, viewer.screenshot())
viewer.close()

print(f'Creating mp4 glimpse ID: {acq_ID, ID}')
### now compile into mp4
mp4_fn = os.path.join(basedir, f'glimpse_{ID}.mp4')
### compile mp4
compile_mp4(glimpse_seq_basedir, mp4_fn, 3, fileformat = '.tiff')


## create sequence of graph images
print(f'Creating graph image sequence ID: {acq_ID, ID}')
# create graph output folder
glimpse_graph_seq_basedir = os.path.join(basedir, f'{ID}_glimpse_graph_seq')
Path(glimpse_graph_seq_basedir).mkdir(parents = True, exist_ok = True)

### create graph by interpolating missing y values
y = gaussian_filter1d(sc_df['Intracellular Mtb content'].interpolate().values, 
                      sigma = 1.5)
y2 = gaussian_filter1d(sc_df['Eccentricity'].interpolate().values, 
                       sigma = 1.5)
y3 = gaussian_filter1d(sc_df['Area'].interpolate().values/sc_df['Area'].interpolate().values.max(), 
                       sigma = 1.5)
x = sc_df['Time (hours)'].values

### iterate over time points
for n, row in tqdm(enumerate(sc_df.iterrows()), total = len(sc_df), 
                   desc = f'Saving graph image sequence ID: {acq_ID, ID}'):
    ### select portion of data to plot up to current time point iteration
    plot_x = x[0:n+1]
    plot_y = y[0:n+1]
    plot_y2 = y2[0:n+1]
    plot_y3 = y3[0:n+1]
    ### get time ID for labelling purposes
    t = row[1]['Time (hours)']     
    ### do not display plot in ipython
    plt.ioff()
    ### initiate plot with two components
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14,5), gridspec_kw={'width_ratios': [1, 1]})
    ### label fn with time point
    glimpse_seq_fn = os.path.join(glimpse_seq_basedir, f'glimpse_{ID}_t_{t}.tiff')
    ### load glimpse image from this timepoint
    glimpse_img = imread(glimpse_seq_fn)
    ### add glimpse frame to first part of subplot
    ax1.imshow(glimpse_img)
    ### turn axis off for glimpse
    ax1.axis('off')
    ### plot portion of graph 
    ax2.plot(plot_x, plot_y, c = palette[0],  label= 'Mtb')
    ax3 = ax2.twinx()
    ax3.plot(plot_x, plot_y2, c = palette[4], label = 'Eccentricity')
    ax3.plot(plot_x, plot_y3, c = palette[5], label = 'Area')
    ### label graph properly 
    ax2.set(xlabel = 'Time (hours)', 
            ylabel = f'Mtb content of cell ID: {ID} \n (raw fluorescence intensity)', 
            ylim=(y.min()*0.85,y.max()*1.15), 
            xlim = (0,74))
    ax3.set(ylabel = f'Relative area and eccentricity change', 
            ylim=(0,1), 
           )
    ax2.legend(loc = 'lower left', fancybox = True, 
                                    framealpha = 0.8, 
#                                     edgecolor = 'cyan'
              )
    ax3.legend(loc = 'lower right', fancybox = True, 
                                    framealpha = 0.8, 
#                                     edgecolor = 'cyan'
              )
    ### despine plot
    sns.despine(right = False, offset=10, trim=False)
    ### new fn for joint glimpse plot
    new_graph_fn = os.path.join(glimpse_graph_seq_basedir, f'glimpse_graph_{ID}_t_{t}.tiff')
    plt.savefig(new_graph_fn, bbox_inches = 'tight', dpi = 314)
## Compile single frame graphs into stack for animation
print(f'Creating graph and glimpse image sequence ID: {acq_ID, ID}')
### now compile into mp4
glimpse_graph_fn = os.path.join(basedir, f'glimpse_graph_{ID}.mp4')
compile_mp4(glimpse_graph_seq_basedir, glimpse_graph_fn, 3,)# fileformat = '.tiff')


Creating glimpse image sequence ID: ((3, 8), 81)


/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari_tools_menu/__init__.py:194: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


Saving glimpse image sequence ID: ((3, 8), 81):   0%|          | 0/75 [00:00<?, ?it/s]

Creating mp4 glimpse ID: ((3, 8), 81)
Creating graph image sequence ID: ((3, 8), 81)


Saving graph image sequence ID: ((3, 8), 81):   0%|          | 0/75 [00:00<?, ?it/s]

/tmp/ipykernel_330135/277814232.py:77: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14,5), gridspec_kw={'width_ratios': [1, 1]})


Creating graph and glimpse image sequence ID: ((3, 8), 81)


# Iteratively generate glimpses

In [ ]:
for assay_info in tqdm(assay_layout.iterrows(), 
                       total = len(assay_layout), 
                       desc = 'Iterating over different assays, generating two extreme glimpses per assay'):
    ### pull acq ID from assay info
    acq_ID = assay_info[0]

    ### iterate over different experiments
    selected_expt_df = df[(df['Acquisition ID'] == acq_ID)]

    ### get extreme cells
    extreme_cases = pd.concat([selected_expt_df[(selected_expt_df['Mean Mtb content'] == np.max(selected_expt_df['Mean Mtb content']))],
                               selected_expt_df[(selected_expt_df['Mean Mtb content'] == np.min(selected_expt_df['Mean Mtb content']))]])
    
    ### iterate over two extreme cases in either
    for ID in tqdm(extreme_cases['Cell ID'].unique(), 
                   desc = f'Iterating over two extreme cells in acquisition ID:{acq_ID}'):
        
        ### isolate a single cell ID from df of interesting cells
        sc_df = extreme_cases[extreme_cases['Cell ID']==ID]
        
        ### use acq ID to pre load correct images
        row, column = acq_ID
        images = tile.compile_mosaic(
                                     image_dir, 
                                     metadata, 
                                     row, 
                                     column, 
                                     set_plane = 'sum_proj',
                                     ).astype(np.uint16)
        
        ### create base dirname
        basedir = f'/mnt/DATA/macrohet/results/glimpses/labelled/multi-plot/{acq_ID}/{ID}/'
        Path(os.path.dirname(basedir)).mkdir(parents=True, exist_ok=True)
        
        # ### make glimpse stack of images
        glimpse_stack = tools.create_glimpse_from_sc_df(sc_df, acq_ID, ID, images)
        
        ### load glimpse into memory
        print(f'Loading glimpse stack {acq_ID, ID} into memory (can take several minutes)')
        glimpse_stack = glimpse_stack.compute().compute()

        ### make glimpse stack of masks
        print(f'Creating mask glimpse ID: {acq_ID, ID}')
        
        ### load masks 
        masks = segmentation_dict[(row, column)]
        
        ### generate mask glimpse
        mask_glimpse_stack, mask_shapes = create_mask_glimpse_from_sc_df(sc_df, acq_ID, ID, masks,)
        
        ### get expt info
        expt_info = sc_df['Strain'].iloc[0],sc_df['Compound'].iloc[0],sc_df['Concentration'].iloc[0], sc_df['Cell ID'].iloc[0]
        expt_info = ','.join(map(str, expt_info))
        
        ### create a set of points to anchor labels at
        label_points = [[t, 0, 500] for t in sc_df['Time (hours)']]
        
        ### create labels
        text_overlay = f'cell ID:{acq_ID[0], acq_ID[1], ID}'
        fixed_labels = {'macrophage':['Macrophage:green' for i in range(len(mask_shapes))],
                        'mtb':['Mtb:magenta' for i in range(len(mask_shapes))],
                        'mask':[f'Mask:cyan' for i in range(len(mask_shapes))],
                        'info':[f'ID:{expt_info}' for i in range(len(mask_shapes))]}
        
        ### create lists of labels for tidier iteration of adding labels 
        labels_list = list([fixed_labels['macrophage'], fixed_labels['mtb'], fixed_labels['mask'], fixed_labels['info']])
        offset_list = list([[20,-5], [40,-5], [60,-5], [80,-5]])
        colour_list = list(['#39FF14', 'magenta', 'cyan', 'white'])

        ### create output directory
        glimpse_seq_basedir = os.path.join(basedir, f'{ID}_glimpse_seq')
        Path(glimpse_seq_basedir).mkdir(parents=True, exist_ok=True)
        
        ### load glimpse into napari
        viewer = napari.Viewer()
        viewer.add_image(mask_glimpse_stack, scale = napari_scale, colormap='cyan')
        viewer.add_image(glimpse_stack, channel_axis = 0, colormap= ['green', 'magenta'], blending = ['additive', 'additive'],
                         scale = napari_scale)
        
        ### add scale bar and time bar
        viewer.theme = 'light'
        viewer.scale_bar.visible = True
        viewer.scale_bar.unit = 'm'
        viewer.scale_bar.font_size = text_size
        viewer.text_overlay.visible = True
        viewer.text_overlay.color = 'black'
        viewer.text_overlay.position = 'bottom_left'
        viewer.text_overlay.font_size = text_size
        viewer.dims.events.current_step.connect(update_slider)
        
        ### add three sets of labels
        for config in zip(labels_list, offset_list, colour_list):
            string, offset, colour = config
            viewer.add_points(
            label_points, 
            scale = [1, napari_scale[0], napari_scale[1]],
            face_color = 'transparent', 
            edge_color = 'transparent', 
            text = {'string':string,
                    'anchor': 'upper_right',
                    'translation': offset,
                    'size': text_size,
                    'color': colour}, )
        
        ### save image sequence
        for t in tqdm(list(sc_df['Time (hours)']), desc = f'Saving glimpse image sequence ID: {acq_ID, ID}'):
            viewer.dims.current_step = (t, cam_coords[-2], cam_coords[-1])
            viewer.camera.zoom = zoom*0.95
            glimpse_seq_fn = os.path.join(glimpse_seq_basedir, f'glimpse_{ID}_t_{t}.tiff')
            imsave(glimpse_seq_fn, viewer.screenshot())
        viewer.close()

        ### now compile into mp4
        print(f'Creating mp4 glimpse ID: {acq_ID, ID}')
        mp4_fn = os.path.join(basedir, f'glimpse_{ID}.mp4')
        
        ### compile mp4
        compile_mp4(glimpse_seq_basedir, mp4_fn, 3, fileformat = '.tiff')

        ## create sequence of graph images
        print(f'Creating graph image sequence ID: {acq_ID, ID}')
        
        # create graph output folder
        glimpse_graph_seq_basedir = os.path.join(basedir, f'{ID}_glimpse_graph_seq')
        Path(glimpse_graph_seq_basedir).mkdir(parents = True, exist_ok = True)

        ### create graph by interpolating missing y values
        y = gaussian_filter1d(sc_df['Intracellular Mtb content'].interpolate().values, 
                              sigma = 1.5)
        y2 = gaussian_filter1d(sc_df['Eccentricity'].interpolate().values, 
                               sigma = 1.5)
        y3 = gaussian_filter1d(sc_df['Area'].interpolate().values/sc_df['Area'].interpolate().values.max(), 
                               sigma = 1.5)
        x = sc_df['Time (hours)'].values

        ### iterate over time points
        for n, row in tqdm(enumerate(sc_df.iterrows()), total = len(sc_df), 
                           desc = f'Saving graph image sequence ID: {acq_ID, ID}'):
            
            ### select portion of data to plot up to current time point iteration
            plot_x = x[0:n+1]
            plot_y = y[0:n+1]
            plot_y2 = y2[0:n+1]
            plot_y3 = y3[0:n+1]
            
            ### get time ID for labelling purposes
            t = row[1]['Time (hours)']     
            
            ### do not display plot in ipython
            plt.ioff()
            
            ### initiate plot with two components
            fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14,5), gridspec_kw={'width_ratios': [1, 1]})
            
            ### label fn with time point
            glimpse_seq_fn = os.path.join(glimpse_seq_basedir, f'glimpse_{ID}_t_{t}.tiff')
            
            ### load glimpse image from this timepoint
            glimpse_img = imread(glimpse_seq_fn)
            
            ### add glimpse frame to first part of subplot
            ax1.imshow(glimpse_img)
            
            ### turn axis off for glimpse
            ax1.axis('off')
            
            ### plot portion of graph 
            ax2.plot(plot_x, plot_y, c = palette[0],  label= 'Mtb')
            ax3 = ax2.twinx()
            ax3.plot(plot_x, plot_y2, c = palette[4], label = 'Eccentricity')
            ax3.plot(plot_x, plot_y3, c = palette[5], label = 'Area')
            
            ### label graph properly 
            ax2.set(xlabel = 'Time (hours)', 
                    ylabel = f'Mtb content of cell ID: {ID} \n (raw fluorescence intensity)', 
                    ylim=(y.min()*0.85,y.max()*1.15), 
                    xlim = (0,74))
            ax3.set(ylabel = f'Relative area and eccentricity change', 
                    ylim=(0,1), 
                   )
            ax2.legend(loc = 'lower left', fancybox = True, 
                                            framealpha = 0.8, 
                      )
            ax3.legend(loc = 'lower right', fancybox = True, 
                                            framealpha = 0.8, 
                      )
            
            ### despine plot
            sns.despine(right = False, offset=10, trim=False)
            
            ### new fn for joint glimpse plot
            new_graph_fn = os.path.join(glimpse_graph_seq_basedir, f'glimpse_graph_{ID}_t_{t}.tiff')
            plt.savefig(new_graph_fn, bbox_inches = 'tight', dpi = 314)
        
        ## Compile single frame graphs into stack for animation
        print(f'Creating graph and glimpse image sequence ID: {acq_ID, ID}')
        
        ### now compile into mp4
        glimpse_graph_fn = os.path.join(basedir, f'glimpse_graph_{ID}.mp4')
        compile_mp4(glimpse_graph_seq_basedir, glimpse_graph_fn, 3,)# fileformat = '.tiff')


Iterating over different assays, generating two extreme glimpses per assay:   0%|          | 0/24 [00:00<?, ?i…

Iterating over two extreme cells in acquisition ID:(3, 4):   0%|          | 0/2 [00:00<?, ?it/s]

Creating glimpse ID: ((3, 4), 413):   0%|          | 0/75 [00:00<?, ?it/s]

/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/dask/array/core.py:1744: PerformanceWarning: Increasing number of chunks by factor of 16
  return da_func(*args, **kwargs)


Loading glimpse stack ((3, 4), 413) into memory (can take several minutes)
Creating mask glimpse ID: ((3, 4), 413)


...

/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/dask/array/core.py:1744: PerformanceWarning: Increasing number of chunks by factor of 29
  return da_func(*args, **kwargs)


Loading glimpse stack ((3, 6), 72) into memory (can take several minutes)
Creating mask glimpse ID: ((3, 6), 72)


Creating mask glimpse ID: ((3, 6), 72):   0%|          | 0/75 [00:00<?, ?it/s]

/tmp/ipykernel_330135/3847468708.py:45: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  mask_shapes = np.asarray(coords_stack, )#axis = 0)
/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari_tools_menu/__init__.py:194: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


Saving glimpse image sequence ID: ((3, 6), 72):   0%|          | 0/75 [00:00<?, ?it/s]